In [1]:
import kagglehub

path = kagglehub.dataset_download("mathurinache/the-lj-speech-dataset")

print("Path to dataset files:", path)

/Users/domenicodemarchis/Desktop/IAARTS/classifier/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/domenicodemarchis/Desktop/IAARTS/classifier/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 2.99G/2.99G [05:16<00:00, 10.1MB/s]

Extracting files...


Path to dataset files: /Users/domenicodemarchis/.cache/kagglehub/datasets/mathurinache/the-lj-speech-dataset/versions/1


In [4]:
path = kagglehub.dataset_download("chrisfilo/demand")

print("Path to dataset files:", path)

Resuming download from 7174356992 bytes (206440174 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/chrisfilo/demand?dataset_version_number=1 (7174356992/7380797166) bytes left.


100%|██████████| 6.87G/6.87G [00:10<00:00, 20.1MB/s]

Extracting files...


Path to dataset files: /Users/domenicodemarchis/.cache/kagglehub/datasets/chrisfilo/demand/versions/1


In [8]:
path = kagglehub.dataset_download("raycarterrete/musdb18-stem-division-vocals-and-instrumentals")

print("Path to dataset files:", path)

100%|██████████| 1.14G/1.14G [00:35<00:00, 34.4MB/s]

Extracting files...


Path to dataset files: /Users/domenicodemarchis/.cache/kagglehub/datasets/raycarterrete/musdb18-stem-division-vocals-and-instrumentals/versions/1


In [9]:
import os
import shutil

datasets = ["mathurinache", "chrisfilo", "raycarterrete"]

src_path = os.path.join(os.path.expanduser("~"), ".cache/kagglehub/datasets")
sources = [os.path.join(src_path,src) for src in datasets]

dst = os.path.abspath("datasets")

for src in sources:
    try:
        shutil.move(src, dst)
        print(f"Spostata {src} → {dst}")
    except shutil.Error as e:
        print(f"Errore durante lo spostamento: {e}")

Errore durante lo spostamento: Destination path '/Users/domenicodemarchis/Desktop/IAARTS/classifier/datasets/mathurinache' already exists
Errore durante lo spostamento: Destination path '/Users/domenicodemarchis/Desktop/IAARTS/classifier/datasets/chrisfilo' already exists
Spostata /Users/domenicodemarchis/.cache/kagglehub/datasets/raycarterrete → /Users/domenicodemarchis/Desktop/IAARTS/classifier/datasets


In [10]:
from pydub import AudioSegment
from tqdm.notebook import tqdm

from_dirs = ["NPARK_48k/NPARK", "PCAFETER_48k/PCAFETER", "PSTATION_48k/PSTATION", "STRAFFIC_48k/STRAFFIC", "TMETRO_48k/TMETRO"]

to_dirs = ["park_noises", "cafeteria_noises", "station_noises", "traffic_noises", "metro_noises"]


os.makedirs("datasets/background_simplified", exist_ok = True)

input_dirs  = [os.path.join(os.path.abspath("datasets/chrisfilo/demand/versions/1"), dir) for dir in from_dirs]
output_dirs  = [os.path.join(os.path.abspath("datasets/background_simplified"), dir) for dir in to_dirs]

for od in output_dirs: os.makedirs(od, exist_ok = True)

start_ms = (1 * 60 + 30) * 1000
end_ms   = (2 * 60 + 30) * 1000 

for input_dir, output_dir in zip(input_dirs, output_dirs):
    for fname in os.listdir(input_dir):
        if not fname.lower().endswith(".wav"): continue

        path_file = os.path.join(input_dir,fname)
        audio = AudioSegment.from_wav(path_file)

        if len(audio) < end_ms:
            print(f"{fname}: durata inferiore a 2:30, skip.")
            continue

        slice_audio = audio[start_ms:end_ms]
        
        tag = os.path.basename(output_dir)
        base, _    = os.path.splitext(fname)
        out_name   = f"{base}_{tag}_simplified.wav"
        
        path_out   = os.path.join(output_dir, out_name)
        slice_audio.export(path_out, format="wav")

In [21]:
def convert_file(input_path: str, output_path: str, sample_rate: int = 44100, channels: int = 2):
    
    audio = AudioSegment.from_file(input_path)

    audio = audio.set_frame_rate(sample_rate).set_channels(channels)

    audio.export(output_path, format="wav")
    print(f"Converted: {input_path} -> {output_path}")

music_set_path = os.path.abspath("datasets/raycarterrete/musdb18-stem-division-vocals-and-instrumentals/versions/1/stem_files")
mp4_files = [f for f in os.listdir(music_set_path)]

os.makedirs("datasets/music_set_wav", exist_ok = True)
os.makedirs("datasets/music_set_wav/instrumentals", exist_ok=True)
os.makedirs("datasets/music_set_wav/vocals", exist_ok=True)

target_root_vocals = os.path.abspath("datasets/music_set_wav/vocals")
target_root_instrumentals = os.path.abspath("datasets/music_set_wav/instrumentals")

for file in mp4_files:
    input_path = os.path.join(music_set_path, file)
    output_file, _ = os.path.splitext(file)
    output_file += '.wav'
    
    if "instrumental" in output_file: target_root = target_root_instrumentals
    else: target_root = target_root_vocals
    
    output_path = os.path.join(target_root, output_file)
    
    try:
        convert_file(input_path, output_path, sample_rate=44100, channels=2)
    except Exception as e:
        print(f"Errore nella conversione di {input_path}: {e}")

Converted: /Users/domenicodemarchis/Desktop/IAARTS/classifier/datasets/raycarterrete/musdb18-stem-division-vocals-and-instrumentals/versions/1/stem_files/Leaf - Summerghost.stem_vocals.mp4 -> /Users/domenicodemarchis/Desktop/IAARTS/classifier/datasets/music_set_wav/vocals/Leaf - Summerghost.stem_vocals.wav
Converted: /Users/domenicodemarchis/Desktop/IAARTS/classifier/datasets/raycarterrete/musdb18-stem-division-vocals-and-instrumentals/versions/1/stem_files/North To Alaska - All The Same.stem_vocals.mp4 -> /Users/domenicodemarchis/Desktop/IAARTS/classifier/datasets/music_set_wav/vocals/North To Alaska - All The Same.stem_vocals.wav
Converted: /Users/domenicodemarchis/Desktop/IAARTS/classifier/datasets/raycarterrete/musdb18-stem-division-vocals-and-instrumentals/versions/1/stem_files/Tim Taler - Stalker.stem_instrumental.mp4 -> /Users/domenicodemarchis/Desktop/IAARTS/classifier/datasets/music_set_wav/instrumentals/Tim Taler - Stalker.stem_instrumental.wav
Converted: /Users/domenicodemar

# Nota Bene
Il dataset 'music_set_wav' convertito contiene instrumental e vocals, per un'applicazione successiva ci serve ricostruire la canzone

In [25]:
import os
from pathlib import Path
import librosa
import numpy as np
import soundfile as sf
from tqdm.notebook import tqdm

vocal_dir        = Path("datasets/music_set_wav/vocals")
instrumental_dir = Path("datasets/music_set_wav/instrumentals")
output_dir       = Path("datasets/music_set_wav/complete_song")
output_dir.mkdir(parents=True, exist_ok=True)

def build_map(directory: Path, suffix: str):
    m = {}
    for p in directory.iterdir():
        name = p.name
        if name.endswith(suffix):
            title = name.removesuffix(suffix)
            m[title] = p
    return m

voc_map  = build_map(vocal_dir,        ".stem_vocals.wav")
inst_map = build_map(instrumental_dir, ".stem_instrumental.wav")

common_titles = sorted(set(voc_map) & set(inst_map))

for title in tqdm(common_titles, desc="Reconstructing Songs"):
    voc_file  = voc_map[title]
    inst_file = inst_map[title]

    voc,  sr1 = librosa.load(voc_file,  sr=None, mono=True)
    inst, sr2 = librosa.load(inst_file, sr=None, mono=True)
    if sr1 != sr2:
        print(f"[WARN] Sample rate mismatch in '{title}': {sr1} vs {sr2}, skipping.")
        continue

    min_len = min(len(voc), len(inst))
    voc, inst = voc[:min_len], inst[:min_len]

    mixed = voc + inst
    mixed /= np.max(np.abs(mixed))

    out_path = output_dir / f"{title}.wav"
    sf.write(str(out_path), mixed, sr1, format="WAV")

Reconstructing Songs:   0%|          | 0/93 [00:00<?, ?it/s]